<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao23_generate_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geração de Texto Literário com LSTM e Word2Vec Pré-treinado

Este notebook demonstra como treinar uma rede LSTM (Long Short-Term Memory) para gerar texto literário em inglês.

O principal diferencial aqui é o uso de embeddings de palavras pré-treinados, especificamente o **Word2Vec** (treinado no Google News), para inicializar a camada de Embedding da nossa rede. Isso permite que o modelo já comece com um "entendimento" semântico das palavras, acelerando o treinamento e melhorando a qualidade do texto gerado, especialmente com datasets menores.

## 1. Instalação e Imports

Primeiro, importamos as bibliotecas necessárias. O `gensim` será usado para carregar o modelo Word2Vec.

In [1]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 67.9 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gensim.downloader
import numpy as np
import string
import requests # Para baixar o texto

## 2. Download dos Dados

### a) Corpus de Texto Literário

Vamos usar "Alice's Adventures in Wonderland" de Lewis Carroll, disponível no Projeto Gutenberg.

In [3]:
url = "https://www.gutenberg.org/files/11/11-0.txt"
response = requests.get(url)
data = response.text

# Salvar uma cópia local (opcional)
with open("alice_wonderland.txt", "w", encoding="utf-8") as f:
    f.write(data)

print(f"Texto baixado. Tamanho: {len(data)} caracteres.")
print("--- Início do Texto ---")
print(data[500:1000])
print("-----------------------")

Texto baixado. Tamanho: 144696 caracteres.
--- Início do Texto ---
 Mock Turtle’s Story
 CHAPTER X.     The Lobster Quadrille
 CHAPTER XI.    Who Stole the Tarts?
 CHAPTER XII.   Alice’s Evidence




CHAPTER I.
Down the Rabbit-Hole


Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into
the book her sister was reading, but it had no pictures or
conversations in it, “and what is the use of a book,” thought Alice
“without pictures or conversations?”

So she was considering in her
-----------------------


### b) Modelo Word2Vec Pré-treinado

Vamos usar o `gensim.downloader` para carregar o modelo Word2Vec clássico treinado no Google News.

**AVISO:** Este modelo é grande (1.5GB) e o download/carregamento pode levar alguns minutos.

In [4]:
# Nome do modelo: 'word2vec-google-news-300'
print("Carregando modelo Word2Vec (word2vec-google-news-300)...")
print("Isso pode levar alguns minutos...")
word2vec_model = gensim.downloader.load('word2vec-google-news-300')

embedding_dim = word2vec_model.vector_size
print(f"\nModelo Word2Vec carregado com sucesso!")
print(f"Dimensão do Embedding: {embedding_dim}")

# Teste rápido para ver se funciona
try:
    print("\nTeste de similaridade (king - man + woman):")
    similar = word2vec_model.most_similar(positive=['king', 'woman'], negative=['man'], topn=1)
    print(f"Resultado: {similar}")
except KeyError as e:
    print(f"Erro no teste de similaridade: {e}. (Alguma palavra pode estar faltando)")

Carregando modelo Word2Vec (word2vec-google-news-300)...
Isso pode levar alguns minutos...
[==================================================] 100.0% 1662.8/1662.8MB downloaded

Modelo Word2Vec carregado com sucesso!
Dimensão do Embedding: 300

Teste de similaridade (king - man + woman):
Resultado: [('queen', 0.7118193507194519)]


## 3. Pré-processamento do Texto

Agora, vamos limpar o texto e transformá-lo em sequências que a LSTM possa entender.

In [5]:
def clean_text(text):
    # Remove cabeçalhos/rodapés do Gutenberg (aproximação)
    start_marker = "*** START OF THIS PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***"
    end_marker = "*** END OF THIS PROJECT GUTENBERG EBOOK ALICE'S ADVENTURES IN WONDERLAND ***"
    try:
        start_index = text.index(start_marker) + len(start_marker)
        end_index = text.index(end_marker)
        text = text[start_index:end_index]
    except ValueError:
        print("Marcadores do Gutenberg não encontrados. Usando o texto completo.")

    # Converte para minúsculas
    text = text.lower()

    # Remove pontuação e números
    translator = str.maketrans('', '', string.punctuation + string.digits)
    text = text.translate(translator)

    # Remove quebras de linha e espaços extras
    text = text.replace('\r\n', ' ').replace('\n', ' ')
    text = ' '.join(text.split()) # Remove espaços múltiplos
    return text

cleaned_data = clean_text(data)
tokens = cleaned_data.split()

print(f"Total de tokens no corpus limpo: {len(tokens)}")
print(f"Exemplo de tokens: {tokens[100:110]}")

Marcadores do Gutenberg não encontrados. Usando o texto completo.
Total de tokens no corpus limpo: 26476
Exemplo de tokens: ['tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', 'bank', 'and']


In [6]:
# Criar sequências de N palavras para prever a (N+1)-ésima
# Ex: [palavra1, palavra2, ..., palavra50] -> [palavra51]

seq_length = 50 # Usar 50 palavras anteriores para prever a próxima
sequences = []

for i in range(seq_length, len(tokens)):
    # Pega 50 palavras + 1 palavra alvo
    seq = tokens[i-seq_length:i+1]
    sequences.append(' '.join(seq))

print(f"Total de sequências de treinamento: {len(sequences)}")
print(f"\nExemplo de sequência (como texto):\n{sequences[0]}")

Total de sequências de treinamento: 26426

Exemplo de sequência (como texto):
start of the project gutenberg ebook illustration alice’s adventures in wonderland by lewis carroll the millennium fulcrum edition contents chapter i down the rabbithole chapter ii the pool of tears chapter iii a caucusrace and a long tale chapter iv the rabbit sends in a little bill chapter v advice from


In [7]:
# Tokenização com o Keras Tokenizer
# Isso criará o mapeamento palavra -> índice
tokenizer = Tokenizer(oov_token='<oov>') # <oov> para palavras fora do vocabulário
tokenizer.fit_on_texts(sequences)

word_index = tokenizer.word_index
total_words = len(word_index) + 1 # +1 para o índice 0 (que é reservado)

print(f"Tamanho total do vocabulário (do nosso corpus): {total_words}")

Tamanho total do vocabulário (do nosso corpus): 3489


In [8]:
# Converter as sequências de texto em sequências de inteiros (índices)
input_sequences_int = tokenizer.texts_to_sequences(sequences)
input_sequences = np.array(input_sequences_int)

# Separar X (features) e y (label)
X = input_sequences[:, :-1]  # Todas as palavras, exceto a última
y = input_sequences[:, -1]   # Apenas a última palavra

# Nosso 'y' são índices inteiros. Para usar 'categorical_crossentropy',
# precisaríamos de one-hot encoding.
# Alternativa: Usar 'sparse_categorical_crossentropy' que aceita 'y' como índices.
# Vamos usar sparse_categorical_crossentropy.

max_sequence_len = X.shape[1] # Deve ser igual a 'seq_length'

print(f"Shape de X (input): {X.shape}")
print(f"Shape de y (target): {y.shape}")
print(f"Tamanho máximo da sequência de entrada: {max_sequence_len}")

Shape de X (input): (26426, 50)
Shape de y (target): (26426,)
Tamanho máximo da sequência de entrada: 50


## 4. Criação da Matriz de Embedding

Este é o passo crucial. Vamos criar uma matriz onde a linha `i` contém o vetor Word2Vec para a palavra de índice `i` do *nosso* vocabulário (do tokenizer).

In [9]:
# Inicializar a matriz de embedding com zeros
# Shape: (total_words, embedding_dim)
embedding_matrix = np.zeros((total_words, embedding_dim))

words_in_vocab = 0
words_not_in_vocab = 0

# Iterar sobre o nosso vocabulário (word_index do Keras Tokenizer)
for word, i in word_index.items():
    if word in word2vec_model:
        # Se a palavra existe no Word2Vec, pegamos o vetor
        embedding_matrix[i] = word2vec_model[word]
        words_in_vocab += 1
    else:
        # Se não, o vetor permanece como zeros (ou poderia ser aleatório)
        words_not_in_vocab += 1

print(f"Matriz de embedding criada com shape: {embedding_matrix.shape}")
print(f"Palavras do nosso vocabulário encontradas no Word2Vec: {words_in_vocab}")
print(f"Palavras não encontradas (OOV) no Word2Vec: {words_not_in_vocab}")

Matriz de embedding criada com shape: (3489, 300)
Palavras do nosso vocabulário encontradas no Word2Vec: 2331
Palavras não encontradas (OOV) no Word2Vec: 1157


## 5. Construção do Modelo LSTM

Definimos a arquitetura da rede. A camada `Embedding` será inicializada com a `embedding_matrix` que acabamos de criar e será **congelada** (`trainable=False`) para que o treinamento não altere os pesos do Word2Vec.

In [10]:
model = Sequential()

# Camada de Embedding
model.add(Embedding(
    input_dim=total_words,       # Tamanho do vocabulário
    output_dim=embedding_dim,  # Dimensão do Word2Vec (300)
    weights=[embedding_matrix],  # Pesos pré-treinados
    input_length=max_sequence_len, # Tamanho da sequência de entrada (50)
    trainable=False              # Congela os pesos do embedding
))

# Camadas LSTM
# return_sequences=True é necessário se a próxima camada for outra LSTM
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100)) # Removed return_sequences=True
model.add(Dropout(0.2))

# Camada de Saída
# A saída é uma probabilidade para cada palavra do vocabulário
model.add(Dense(total_words, activation='softmax'))

# Compilação
# Usamos 'sparse_categorical_crossentropy' pois nosso 'y' são índices inteiros
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     1,046,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,046,700 (3.99 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,046,700 (3.99 MB)

## 6. Treinamento do Modelo

Vamos treinar o modelo. Com um corpus pequeno como "Alice", 20-30 épocas podem ser suficientes para um resultado razoável. Para um modelo de produção, seriam necessárias muito mais épocas e dados.

In [21]:
print("Iniciando o treinamento...")
# Aumente o número de épocas para melhores resultados (ex: 100)
history = model.fit(X, y, epochs=200, batch_size=128, verbose=1)

Iniciando o treinamento...
Epoch 1/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5415 - loss: 1.7668
Epoch 2/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5398 - loss: 1.7744
Epoch 3/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.5468 - loss: 1.7568
Epoch 4/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.5457 - loss: 1.7493
Epoch 5/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.5503 - loss: 1.7520
Epoch 6/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5351 - loss: 1.7756
Epoch 7/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5425 - loss: 1.7628
Epoch 8/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.5402 - loss: 1.7558
Epoch 9/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5484 - loss: 1.7433
Epoch 10/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5384 - loss: 1.7551
Epoch 11/200
207/207 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5474 - loss: 1.7475
Epo

## 7. Geração de Texto

Agora, a parte divertida. Vamos criar uma função para gerar texto usando o modelo treinado.

Usaremos uma técnica chamada **"Temperature Sampling"** para controlar a "criatividade" do modelo.
* `temperature` baixa (ex: 0.2): Texto mais previsível e conservador.
* `temperature` alta (ex: 1.0): Texto mais criativo e arriscado (pode gerar mais erros).

In [22]:
def sample(preds, temperature=1.0):
    """Helper para re-ponderar a distribuição de probabilidade com 'temperatura'."""
    # Adiciona 1e-7 para evitar log(0)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    # Amostra um índice da distribuição multinomial
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer, temperature=0.7):
    generated_text = seed_text
    current_seed = seed_text

    for _ in range(next_words):
        # 1. Tokenizar o texto atual (seed)
        token_list = tokenizer.texts_to_sequences([current_seed])[0]

        # 2. Fazer o padding para ter o tamanho exato de entrada do modelo
        token_list_padded = pad_sequences([token_list], maxlen=max_sequence_len, padding='pre')

        # 3. Obter as predições (probabilidades da próxima palavra)
        predicted_probs = model.predict(token_list_padded, verbose=0)[0]

        # 4. Aplicar o sampling com temperatura para escolher o índice da próxima palavra
        predicted_index = sample(predicted_probs, temperature)

        # 5. Converter o índice de volta para uma palavra
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        # 6. Se a palavra for <oov> ou vazia, pula esta iteração
        if output_word == "<oov>" or not output_word:
            continue

        # 7. Adicionar a nova palavra ao texto gerado
        generated_text += " " + output_word

        # 8. Atualizar o 'current_seed' para a próxima iteração
        # (Remove a primeira palavra e adiciona a nova ao final)
        current_seed = ' '.join(current_seed.split()[1:]) + " " + output_word

    return generated_text

In [23]:
# Vamos pegar um "seed" (semente) aleatório do texto original
seed_start_index = np.random.randint(0, len(sequences))
raw_seed = sequences[seed_start_index]

# O seed text precisa ter exatamente 'seq_length' (50) palavras
seed_text = ' '.join(raw_seed.split()[:-1]) # Pega as primeiras 50 palavras da sequência

print(f"--- SEED INICIAL (Semente) ---\n{seed_text}")

print("\n--- TEXTO GERADO (Temp 0.5 - Mais conservador) ---")
generated_output_05 = generate_text(seed_text, 100, model, max_sequence_len, tokenizer, temperature=0.5)
print(generated_output_05)

print("\n--- TEXTO GERADO (Temp 1.0 - Mais criativo) ---")
generated_output_10 = generate_text(seed_text, 100, model, max_sequence_len, tokenizer, temperature=1.0)
print(generated_output_10)

--- SEED INICIAL (Semente) ---
with a sigh “it’s always teatime and we’ve no time to wash the things between whiles” “then you keep moving round i suppose” said alice “exactly so” said the hatter “as the things get used up” “but what happens when you come to the beginning again” alice ventured to ask

--- TEXTO GERADO (Temp 0.5 - Mais conservador) ---
with a sigh “it’s always teatime and we’ve no time to wash the things between whiles” “then you keep moving round i suppose” said alice “exactly so” said the hatter “as the things get used up” “but what happens when you come to the beginning again” alice ventured to ask “suppose we change the subject” the march hare interrupted yawning “i’m that’s thimble” said the mock turtle “crumbs do you mean of” said the hatter “i didn’t seem it” said the hatter “as you know walk in it” very fine tone though i was now in the gryphon said to itself in the way were on the other side and her as it was peeped and broke in a head in looking at the pigeon 

## 8. Conclusão e Próximos Passos

Conseguimos treinar uma LSTM que usa Word2Vec para gerar texto! O resultado com 30 épocas e um livro pequeno é apenas razoável, mas o processo está correto.

Para melhorar:

1.  **Mais Dados:** Use um corpus muito maior (ex: vários livros do Projeto Gutenberg combinados).
2.  **Mais Treinamento:** Aumente o número de épocas (ex: 100, 200 ou mais).
3.  **Ajuste Fino (Fine-Tuning):** Tente definir `trainable=True` na camada `Embedding` após algumas épocas iniciais, ou desde o início com uma taxa de aprendizado baixa. Isso permite que os pesos do Word2Vec sejam ligeiramente ajustados para o estilo literário do corpus.
4.  **Arquitetura:** Experimente com mais camadas LSTM, `Bidirectional(LSTM(...))` ou `GRU`.